In [2]:
#########calculates flamespeeds for the .cti files at different equivalence ratios. Uses initial guess from the previous model #############################

import cantera as ct
import numpy as np
import pandas as pd
import os
import csv 
print("Running Cantera Version: " + str(ct.__version__))

To = 298
Po = ct.one_atm

gas = ct.Solution('./cantera/RMG_David/chem_annotated.cti')


#vol_frac_list = np.arange(0.5, 1.2, step=0.07)


#can use below to only test one vol_frac
vol_frac_list = [0.095]


results = {}

for i in  range(len(vol_frac_list)):
    try: 
        
        tol_ss = [1.0e-13, 1.0e-9]  #abs and rel tolerances for steady state problem
        tol_ts = [1.0e-13, 1.0e-9]  #abs and rel tie tolernces for time step function
        
        x = vol_frac_list[i]
        norm_ox = (1-x)*.21
        
        
        print(f'****************************starting new volume fraction: {x}**************************')

        vol_frac_dict = {'CH4(3)': (x/norm_ox), 'O2(4)':((1-x)*.21)/norm_ox, 'N2':((1-x)*0.79)/norm_ox}
        print(vol_frac_dict)
        print(f"O2/CH4 ratio = {vol_frac_dict['O2(4)']/vol_frac_dict['CH4(3)']}. Complete combustion takes 2")
        gas.TPX = To, Po, vol_frac_dict
        width = 0.08
        flame = ct.FreeFlame(gas, width=width)
        flame.flame.set_steady_tolerances(default=tol_ss)   #set tolerances
        flame.flame.set_transient_tolerances(default=tol_ts)
        #flame.set_refine_criteria(ratio=3, slope=0.1, curve=0.1) 
        flame.set_refine_criteria(ratio=5, slope=0.25, curve=0.27)
        flame.max_time_step_count = 900
        loglevel = 1 
#         if i!=0:
#             d = f'./data/David_test_{vol_frac_list[i-1]}.csv'
#             if os.path.exists(d):  
#                 arr2 = ct.SolutionArray(gas)
#                 arr2.read_csv(d)
#                 flame.set_initial_guess(data=arr2)
#                 print(' initial guess has been set')
        #"False" stops the calculation from retrying over and over, thanks Chao 
        #flame.solve(loglevel=loglevel, auto=False)
        flame.solve(loglevel=loglevel, auto=True)
        Su = flame.velocity[0]
        results[x] = Su
        sltn = flame.to_solution_array()
        df1 = sltn.to_pandas()
        #edited this here!! index=False
        df1.to_csv(f'./data/David_test_{x}.csv', index=False)
    except Exception as e: 
        print(f'********************passed volume fraction:{vol_frac_list[i]}, error: {e}*************************************')
        pass


vol_fracs = list(results.keys())
flame_speeds = list(results.values())


print("volume fractions are:")
print(vol_fracs)

print("flame speeds are:")
print(flame_speeds)


with open('final_calcs_David.csv', 'w+') as g:
    writers = csv.writer(g)
    writers.writerow(vol_fracs)
    writers.writerow(flame_speeds)

        



Running Cantera Version: 2.5.1
****************************starting new volume fraction: 0.095**************************
{'CH4(3)': 0.4998684556695607, 'O2(4)': 1.0, 'N2': 3.7619047619047623}
O2/CH4 ratio = 2.0005263157894735. Complete combustion takes 2

************ Solving on 8 point grid with energy equation enabled ************

..............................................................................
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0001709      3.453
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0003649      4.854
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     1.218e-05      6.831
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps     0.0002081       5.01
Attempt Newton solution of steady-state problem...    failure. 
Take 10 timesteps      0.005332      2.981
Attempt Newton solution of steady-state problem...


..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [60] point grid(s).

..............................................................................
##############################################################################
Refining grid in flame.
    New points inserted after grid points 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 
    to resolve C2H(23) C2H2(24) C2H3(30) C2H4(31) C3H3(4828) C3H4(2596) C3H4(5216) CH(18) CH2(S)(26) CH2(T)(19) CH2CHO(36) CH2CO(29) CH2OH(34) CH3(20) CH3CHO(37) CH3O(28) CH3OO(201) CHO3(90) H2CC(25) HCCO(22) HCO(16) S(105) S(26591) 
##############################################################################

..............................................................................
Attempt Newton solution of steady-state problem...    success.

Problem solved on [77] point grid(s).

.............................................